In [25]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from math import sqrt

In [3]:
data = pd.read_csv('Data review - Sheet1.csv')

In [4]:
data

,Unnamed: 0,DEPTH (FT AH),SG (Oil Gravity) @ 60/60F,API,"Viscosity @ SC, cSt","Temp (deg F, res)","Viscosity @ RC, cp",Pressure,"Uob, cp","Uo, cp"
0,NaN,6989,0.95,17.8,121.00,126,13.0,2890,14.0,14.0
1,NaN,6989,0.95,17.8,112.60,126,13.0,2890,14.0,14.0
2,NaN,6989,0.95,18.3,127.30,126,14.0,2890,15.0,15.0
3,NaN,6994,0.95,18.0,114.60,126,13.0,2890,14.0,14.0
4,NaN,6994,0.95,17.9,113.90,126,13.0,2890,14.0,14.0
...,...,...,...,...,...,...,...,...,...,...
357,NaN,9820,0.91,23.3,17.32,171,5.1,4200,11.0,25.0
358,NaN,9820,0.92,22.5,20.04,171,5.9,4200,12.0,28.0
359,NaN,10544,0.82,40.2,2.82,191,0.9,4500,2.1,6.7
360,NaN,10135,0.83,38.9,2.86,191,0.9,4500,1.0,1.2


In [8]:
data.columns

Index(['Unnamed: 0', 'DEPTH (FT AH)', 'SG (Oil Gravity) @ 60/60F', 'API',
       'Viscosity @ SC, cSt', 'Temp (deg F, res)', 'Viscosity @ RC, cp',
       'Pressure', 'Uob, cp', 'Uo, cp'],
      dtype='object')

In [10]:
data = data.drop(['Unnamed: 0', 'Uo, cp'], axis = 1)

In [11]:
data

,DEPTH (FT AH),SG (Oil Gravity) @ 60/60F,API,"Viscosity @ SC, cSt","Temp (deg F, res)","Viscosity @ RC, cp",Pressure,"Uob, cp"
0,6989,0.95,17.8,121.00,126,13.0,2890,14.0
1,6989,0.95,17.8,112.60,126,13.0,2890,14.0
2,6989,0.95,18.3,127.30,126,14.0,2890,15.0
3,6994,0.95,18.0,114.60,126,13.0,2890,14.0
4,6994,0.95,17.9,113.90,126,13.0,2890,14.0
...,...,...,...,...,...,...,...,...
357,9820,0.91,23.3,17.32,171,5.1,4200,11.0
358,9820,0.92,22.5,20.04,171,5.9,4200,12.0
359,10544,0.82,40.2,2.82,191,0.9,4500,2.1
360,10135,0.83,38.9,2.86,191,0.9,4500,1.0


In [13]:
#now to find the original eror between Prof's method and machine learning method
error_pre =  mean_squared_error(data["Uob, cp"].values, data["Viscosity @ RC, cp"].values)
error_pre

44.25302099447514

In [27]:
#make a copy of the data, rewrite a Gradient boosting algorithm using Grid search and get the 
#results and use it to do machine learning 
dataset = data.copy()
dataset = dataset.drop("Viscosity @ RC, cp", axis = 1)
y = dataset["Uob, cp"].values
X = dataset.drop('Uob, cp', axis = 1).values
X.shape

(362, 6)

In [33]:
estimator  = GradientBoostingRegressor(n_estimators = 120, random_state= 42)
params  = {'n_estimators': np.arange(40, 300, 10), "learning_rate": np.arange(0.01, 0.1, 0.01), "max_depth" : [2, 3, 4, 5, 6, 7, 8, 9], "min_samples_leaf": np.arange(1, 10), "max_features": [2, 3, 0.5, 'auto','sqrt', 'log2', None]}

In [34]:
gridsearch = GridSearchCV(estimator=estimator, cv=5, param_grid = params, n_jobs = -1, scoring= "neg_mean_squared_error")
gridsearch.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=120,
                                                 n_iter_n...
                         'ma

In [35]:
gridsearch.best_params_

{'learning_rate': 0.060000000000000005,
 'max_depth': 3,
 'max_features': 3,
 'min_samples_leaf': 3,
 'n_estimators': 290}

In [37]:
mean_squared_error(y, gridsearch.predict(X))

0.1266669995766966

In [38]:
gridsearch.predict(X)

array([14.31570935, 13.67934455, 14.989855  , 13.82979202, 13.82979202,
       16.32850455, 11.5275945 , 14.98280909, 14.25134782, 13.71734709,
       16.39094909, 20.14323595, 17.92373567, 20.97256562, 19.70512189,
       27.59802207, 21.44813974, 33.29511178, 21.43306988, 27.37090144,
       28.96725156, 28.44720414, 27.32053805, 26.0620343 , 28.64641993,
       30.41181845, 30.90815054, 26.74487995, 27.13227382, 22.44103327,
       22.49849717, 22.19179741, 23.55573122, 25.72538079, 25.69723157,
       29.61047179, 17.31205374, 18.30653222, 18.4276121 ,  2.84258674,
        3.08279661,  1.96943993,  2.57444684,  2.49462096,  2.88282083,
        2.88282083,  2.71038647,  2.97641206,  3.43003243,  3.61722771,
        3.66741164,  3.50617908,  3.48805455,  1.39219064,  1.32037921,
        1.31415689,  1.33314418,  1.33314418,  1.33660407,  1.33314418,
        1.34625998,  1.33314418,  1.33314418,  1.33314418,  1.33314418,
        1.33314418,  1.1878935 ,  1.40113885,  1.32037921,  1.38